In [2]:
import numpy as np
import scipy.linalg as sl
import scipy.sparse as sp
import scipy.sparse.linalg as spl

# allow printing large matrix without wrapping
import sys 
np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(linewidth=sys.maxsize)

# supress numpy exponential display
np.set_printoptions(suppress=True)

In [3]:
# concatenate two matrices to a bigger matrix by gluing the head and tail
# takes sparse matrix as input
def concatMat(matA, matB):
    sizeA = matA.shape[0]
    sizeB = matB.shape[0]
    
    copyB = sp.csr_matrix.copy(matB)
    copyB[0,0] = 0
    
    expandA = sp.block_diag((matA,sp.bsr_matrix((sizeB-1,sizeB-1))))
    expandB = sp.block_diag((sp.bsr_matrix((sizeA-1,sizeA-1)),copyB))
    
    return expandA + expandB

In [4]:
# concat a list of matrices all at once
def concatMatTuple(matTuple):
    i = 0
    matOut = matTuple[i]
    while i < len(matTuple)-1:
        matOut = concatMat(matOut,matTuple[i+1])
        i += 1
        
    return matOut

In [5]:
# this chunck defines the fundamental sub-matrices in A2
n = 4

a = (2*3**n)**2
b = (3*3**n)**2
c = (6*3**n)**2

start0 = np.array(
    [[a,0,-a,0,0],
     [0,c,-c,0,0],
     [-a,-c,2*a+b+c,-a,-b],
     [0,0,-a,2*a,-a],
     [0,0,-b,-a,2*a+b+c]])

start = sp.csr_matrix(start0)

end0 = np.flip(start)

end = sp.csr_matrix(end0)

# the red blob on the left side
red1 = np.array(
    [[2*a+b+c,-c,-a,0,0,0,0],
     [-c,a+c,0,0,-a,0,0],
     [-a,0,a,0,0,0,0],
     [0,0,0,c,-c,0,0],
     [0,-a,0,-c,2*a+b+c,-a,-b],
     [0,0,0,0,-a,2*a,-a],
     [0,0,0,0,-b,-a,2*a+b+c]])

# sea horse on the left
shl = sp.csr_matrix(red1)

# the red blob on the right side
red2 = np.flip(red1) 

# sea horse on the right
shr = sp.csr_matrix(red2)

blue = np.array(
    [[2*a+b+c,-c,0,0,0,0,0,0,0,0,0,0,0,-a,0],
     [-c,a+c,0,-a,0,0,0,0,0,0,0,0,0,0,0],
     [0,0,c,-c,0,0,0,0,0,0,0,0,0,0,0],
     [0,-a,-c,2*a+b+c,-a,0,-b,0,0,0,0,0,0,0,0],
     [0,0,0,-a,2*a,0,-a,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,2*c,-c,0,-c,0,0,0,0,0,0],
     [0,0,0,-b,-a,-c,2*a+b+c,-a,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,-a,2*a,-a,0,0,0,0,0,0],
     [0,0,0,0,0,-c,0,-a,2*a+b+c,-a,-b,0,0,0,0],
     [0,0,0,0,0,0,0,0,-a,2*a,-a,0,0,0,0],
     [0,0,0,0,0,0,0,0,-b,-a,2*a+b+c,-c,-a,0,0],
     [0,0,0,0,0,0,0,0,0,0,-c,c,0,0,0],
     [0,0,0,0,0,0,0,0,0,0,-a,0,a+c,0,-c],
     [-a,0,0,0,0,0,0,0,0,0,0,0,0,2*a,-a],
     [0,0,0,0,0,0,0,0,0,0,0,0,-c,-a,2*a+b+c]])

whelk = sp.csr_matrix(blue)

green = np.array(
    [[2*a+b+c,-a,-b,0,0,0,0,0,0,0,0,0,0,0],
     [-a,4*a,-a,0,0,0,0,0,0,0,0,0,-a,-a],
     [-b,-a,2*a+b+c,-c,0,-a,0,0,0,0,0,0,0,0],
     [0,0,-c,c,0,0,0,0,0,0,0,0,0,0],
     [0,0,0,0,a,0,-a,0,0,0,0,0,0,0],
     [0,0,-a,0,0,a+c,-c,0,0,0,0,0,0,0],
     [0,0,0,0,-a,-c,2*a+b+c,-a,-b,0,0,0,0,0],
     [0,0,0,0,0,0,-a,2*a,-a,0,0,0,0,0],
     [0,0,0,0,0,0,-b,-a,2*a+b+c,-c,-a,0,0,0],
     [0,0,0,0,0,0,0,0,-c,a+c,0,0,-a,0],
     [0,0,0,0,0,0,0,0,-a,0,a,0,0,0],
     [0,0,0,0,0,0,0,0,0,0,0,c,-c,0],
     [0,-a,0,0,0,0,0,0,0,-a,0,-c,2*a+b+c,-b],
     [0,-a,0,0,0,0,0,0,0,0,0,0,-b,2*a+b+c]])

turbot = sp.csr_matrix(green)

In [6]:
# this chunck defines sub-matrices in A3
c3 = 7

start3 = concatMatTuple((start,shl,whelk))
end3 = np.flip(start3)

shl3 = concatMatTuple((turbot,shl,whelk))
shr3 = np.flip(shl3)

whelk3 = concatMatTuple((turbot,shl,whelk,turbot,whelk,shr,turbot))
# delete the row of the cirtical point
whelk3 = sp.vstack([whelk3[:c3, :],whelk3[c3+1:, :]])
# delete the column of the critical point
whelk3 = sp.hstack([whelk3[:, :c3],whelk3[:, c3+1:]])
# transform the matrix back to csr
whelk3 = sp.csr_matrix(whelk3)
# connect the critical point from the right to the left
whelk3[c3-1,-c3] = -a
whelk3[c3,-c3] = -a
whelk3[-c3,c3-1] = -a
whelk3[-c3,c3] = -a
whelk3[-c3,-c3] = 4*a

turbot3 = concatMatTuple((whelk,shr,turbot,whelk,turbot,shl,whelk))
# deleting the row of the cirtical point
turbot3 = sp.vstack([turbot3[:c3, :],turbot3[c3+1:, :]])
# delete the column of the critical point
turbot3 = sp.hstack([turbot3[:, :c3],turbot3[:, c3+1:]])
# transform the matrix back to csr
turbot3 = sp.csr_matrix(turbot3)
# connect the critical point from the right to the left
turbot3[c3-1,-c3-1] = -a
turbot3[c3,-c3-1] = -a
turbot3[-c3-1,c3-1] = -a
turbot3[-c3-1,c3] = -a
turbot3[-c3-1,-c3-1] = 4*a

print("start3",start3.shape)
print("seahorse3",shl3.shape)
print("whelk3",whelk3.shape)
print("turbot3",turbot3.shape)

start3 (25, 25)
seahorse3 (34, 34)
whelk3 (79, 79)
turbot3 (80, 80)


/usr/local/lib/python3.8/dist-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


In [7]:
# this chunck defines sub-matrices in A4
c4 = shl3.shape[1]-2 + c3

start4 = concatMatTuple((start3,shl3,whelk3))
end4 = np.flip(start4)

shl4 = concatMatTuple((turbot3,shl3,whelk3))
shr4 = np.flip(shl4)

whelk4 = concatMatTuple((turbot3,shl3,whelk3,turbot3,whelk3,shr3,turbot3))
# delete the row of the cirtical point
whelk4 = sp.vstack([whelk4[:c4, :],whelk4[c4+1:, :]])
# delete the column of the critical point
whelk4 = sp.hstack([whelk4[:, :c4],whelk4[:, c4+1:]])
# transform the matrix back to csr
whelk4 = sp.csr_matrix(whelk4)
# connect the critical point from the right to the left
whelk4[c4-1,-c4-2] = -a
whelk4[c4,-c4-2] = -a
whelk4[-c4-2,c4-1] = -a
whelk4[-c4-2,c4] = -a
whelk4[-c4-2,-c4-2] = 4*a

turbot4 = concatMatTuple((whelk3,shr3,turbot3,whelk3,turbot3,shl3,whelk3))
# deleting the row of the cirtical point
turbot4 = sp.vstack([turbot4[:c4, :],turbot4[c4+1:, :]])
# delete the column of the critical point
turbot4 = sp.hstack([turbot4[:, :c4],turbot4[:, c4+1:]])
# transform the matrix back to csr
turbot4 = sp.csr_matrix(turbot4)
# connect the critical point from the right to the left
turbot4[c4-1,-c4-1] = -a
turbot4[c4,-c4-1] = -a
turbot4[-c4-1,c4-1] = -a
turbot4[-c4-1,c4] = -a
turbot4[-c4-1,-c4-1] = 4*a

print("start4",start4.shape)
print("seahorse4",shl4.shape)
print("whelk4",whelk4.shape)
print("turbot4",turbot4.shape)

start4 (136, 136)
seahorse4 (191, 191)
whelk4 (459, 459)
turbot4 (458, 458)


In [8]:
# this chunck defines sub-matrices in A5
c5 = shl4.shape[1]-2 + c4

start5 = concatMatTuple((start4,shl4,whelk4))
end5 = np.flip(start5)

shl5 = concatMatTuple((turbot4,shl4,whelk4))
shr5 = np.flip(shl5)

whelk5 = concatMatTuple((turbot4,shl4,whelk4,turbot4,whelk4,shr4,turbot4))
# delete the row of the cirtical point
whelk5 = sp.vstack([whelk5[:c5, :],whelk5[c5+1:, :]])
# delete the column of the critical point
whelk5 = sp.hstack([whelk5[:, :c5],whelk5[:, c5+1:]])
# transform the matrix back to csr
whelk5 = sp.csr_matrix(whelk5)
# connect the critical point from the right to the left
whelk5[c5-1,-c5-2] = -a
whelk5[c5,-c5-2] = -a
whelk5[-c5-2,c5-1] = -a
whelk5[-c5-2,c5] = -a
whelk5[-c5-2,-c5-2] = 4*a

turbot5 = concatMatTuple((whelk4,shr4,turbot4,whelk4,turbot4,shl4,whelk4))
# deleting the row of the cirtical point
turbot5 = sp.vstack([turbot5[:c5, :],turbot5[c5+1:, :]])
# delete the column of the critical point
turbot5 = sp.hstack([turbot5[:, :c5],turbot5[:, c5+1:]])
# transform the matrix back to csr
turbot5 = sp.csr_matrix(turbot5)
# connect the critical point from the right to the left
turbot5[c5-1,-c5-3] = -a
turbot5[c5,-c5-3] = -a
turbot5[-c5-3,c5-1] = -a
turbot5[-c5-3,c5] = -a
turbot5[-c5-3,-c5-3] = 4*a

print("start5",start5.shape)
print("seahorse5",shl5.shape)
print("whelk5",whelk5.shape)
print("turbot5",turbot5.shape)

start5 (784, 784)
seahorse5 (1106, 1106)
whelk5 (2667, 2667)
turbot5 (2668, 2668)


In [9]:
# this chunck defines sub-matrices in A6
c6 = shl5.shape[1]-2 + c5

start6 = concatMatTuple((start5,shl5,whelk5))
end6 = np.flip(start6)

shl6 = concatMatTuple((turbot5,shl5,whelk5))
shr6 = np.flip(shl6)

whelk6 = concatMatTuple((turbot5,shl5,whelk5,turbot5,whelk5,shr5,turbot5))
# delete the row of the cirtical point
whelk6 = sp.vstack([whelk6[:c6, :],whelk6[c6+1:, :]])
# delete the column of the critical point
whelk6 = sp.hstack([whelk6[:, :c6],whelk6[:, c6+1:]])
# transform the matrix back to csr
whelk6 = sp.csr_matrix(whelk6)
# connect the critical point from the right to the left
whelk6[c6-1,-c6-4] = -a
whelk6[c6,-c6-4] = -a
whelk6[-c6-4,c6-1] = -a
whelk6[-c6-4,c6] = -a
whelk6[-c6-4,-c6-4] = 4*a

turbot6 = concatMatTuple((whelk5,shr5,turbot5,whelk5,turbot5,shl5,whelk5))
# deleting the row of the cirtical point
turbot6 = sp.vstack([turbot6[:c6, :],turbot6[c6+1:, :]])
# delete the column of the critical point
turbot6 = sp.hstack([turbot6[:, :c6],turbot6[:, c6+1:]])
# transform the matrix back to csr
turbot6 = sp.csr_matrix(turbot6)
# connect the critical point from the right to the left
turbot6[c6-1,-c6-3] = -a
turbot6[c6,-c6-3] = -a
turbot6[-c6-3,c6-1] = -a
turbot6[-c6-3,c6] = -a
turbot6[-c6-3,-c6-3] = 4*a

print("start6",start6.shape)
print("seahorse6",shl6.shape)
print("whelk6",whelk6.shape)
print("turbot6",turbot6.shape)

start6 (4555, 4555)
seahorse6 (6439, 6439)
whelk6 (15543, 15543)
turbot6 (15542, 15542)


In [15]:
A2 = concatMatTuple((start3,shl3,whelk3,shr3,end3))
print(A2.shape)

(193, 193)


In [14]:
A2eigenvalues, A2eigenvectors = sl.eigh(A2.toarray())

In [15]:
A2eigenvalues

array([6801806.76513605, 6801806.86026555, 6801806.86026575, 6801806.86193534, 6801806.86193553, 6801806.95706549])

In [269]:
A3 = concatMatTuple((start4,shl4,whelk4,shr4,end4))
print(A3.shape)

(1109, 1109)


In [270]:
A3eigenvalues, A3eigenvectors = sl.eigh(A3.toarray())

In [271]:
A3eigenvalues

array([    0.        ,     0.97618073,     1.9306546 ,     7.03384025,     8.5165335 ,    14.41362151,    14.91703015,    17.83477118,    25.03075084,    29.81286534,    30.29357687,    33.00891977,    45.41892401,    73.92538023,    73.93164051,    74.1707377 ,    92.45792968,   103.000343  ,   103.00852956,   128.26513632,   136.83388718,   138.59204886,   149.71378988,   154.88128577,   160.1709426 ,   161.33521566,   167.16051321,   168.56047357,   169.1268871 ,   193.54492747,   193.54511849,   201.7238317 ,   203.72146859,   206.28716758,   210.97611097,   212.78514805,   219.67995738,   241.81166017,   246.28666931,   264.66820505,   269.26530217,   287.03435677,   325.46225565,   334.97619429,   341.27380259,   356.51416025,   357.62336623,   389.99194983,   390.00036006,   423.71006037,   423.81963602,   423.8196361 ,   424.14139679,   427.92192743,   429.79380627,   434.75629751,   434.88043352,   435.7781819 ,   444.93115718,   508.08215833,   538.25858892,   538.32899701,  

In [21]:
A4 = concatMatTuple((start5,shl5,whelk5,shr5,end5))
print(A4.shape)

(6443, 6443)


In [253]:
A4eigenvalues, A4eigenvectors = sl.eigh(A4.toarray())

In [23]:
A4eigenvalues, A4eigenvectors = spl.eigsh(A4,k=1000,which='SM')

In [254]:
A4eigenvalues

array([     0.        ,      0.62093351,      1.22944026,      4.48073566,      5.39627168,      9.2076707 ,      9.67769223,     11.45347781,     16.01929265,     19.21933239,     19.51755286,     21.22999426,     28.71779898,     47.57928896,     47.59408046,     47.63202445,     59.025969  ,     67.37232663,     67.38064051,     81.42525638,     87.39877564,     88.76895543,     97.14785712,     97.97445546,    102.77288392,    104.08268952,    108.97396032,    109.79888473,    110.56674318,    130.95278797,    130.95302895,    135.02799659,    136.31704132,    137.67847551,    140.69325518,    141.59703766,    147.01020441,    160.27232778,    163.3738551 ,    175.03644163,    178.29750496,    189.26331887,    216.08207446,    223.59947523,    228.20822976,    238.9700888 ,    239.09382312,    269.82130817,    269.82529969,    290.90748681,    292.15483872,    292.15483875,    292.60629284,    292.87806108,    294.91730845,    298.46048838,    298.72718639,    298.78251287,    300.

In [10]:
A5 = concatMatTuple((start6,shl6,whelk6,shr6,end6))
print(A5.shape)

(37527, 37527)


In [11]:
A5eigenvalues, A5eigenvectors = sl.eigh(A5.toarray())

In [0]:
A5eigenvalues

In [128]:
eigvalues, eigvectors = sl.eigh([[1,3],[3,1]])

In [130]:
eigvalues[0]

-2.0

In [132]:
eigvectors[1]

array([0.70710678, 0.70710678])

In [147]:
testA = np.array([[1,2,3],[4,5,6],[7,8,9]])
spA = sp.csr_matrix(testA)
spC = concatMat(spA,spA)
print(spC.toarray())
spC = sp.vstack([spC[:2, :],spC[3:, :]])
print(spC.toarray())
spC = sp.hstack([spC[:, :2],spC[:, 3:]])
print(spC.toarray())
spC = sp.csr_matrix(spC)
spC[1,1] = 77
print(spC.toarray())

[[1. 2. 3. 0. 0.]
 [4. 5. 6. 0. 0.]
 [7. 8. 9. 2. 3.]
 [0. 0. 4. 5. 6.]
 [0. 0. 7. 8. 9.]]
[[1. 2. 3. 0. 0.]
 [4. 5. 6. 0. 0.]
 [0. 0. 4. 5. 6.]
 [0. 0. 7. 8. 9.]]
[[1. 2. 0. 0.]
 [4. 5. 0. 0.]
 [0. 0. 5. 6.]
 [0. 0. 8. 9.]]
[[ 1.  2.  0.  0.]
 [ 4. 77.  0.  0.]
 [ 0.  0.  5.  6.]
 [ 0.  0.  8.  9.]]
